In [1]:
import pandas as pd
import numpy  as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import streamlit as st
import requests
from io import StringIO
import os


# Conectar a la base de datos SQLite
conn = sqlite3.connect(r"C:\Users\DELL\OneDrive\Documentos\kadija\compu\SALUD MENTAL EN LA INDUSTRIA TECNOLÓGICA 1 (4).sqlite")

cur = conn.cursor()

In [2]:
# Consultar las tablas en la base de datos
consulta_ntablas = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"


# Mostrar las entidades
print("Entidades en la base de datos:")
entidades = pd.read_sql_query(sql = consulta_ntablas, con = conn)
entidades

Entidades en la base de datos:


,name
0,Answer
1,Question
2,Respuestas
3,Survey


In [3]:
# Crear consulta para convertir en DF
consulta = "SELECT * FROM Respuestas;"

df_Respuestas = pd.read_sql_query(sql = consulta, con = conn)

conn.close()

In [4]:
# Filtrado del DF por desde 2016-2019 y Estados Unidos como país para realizar la Investigación
User_ID = df_Respuestas[(df_Respuestas['QuestionID'] == 3) & (df_Respuestas['AnswerText'] == 'Estados Unidos')]['UserID'].unique()

df_Investigacion1 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2016)]

df_Investigacion2 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2017)]

df_Investigacion3 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2018)]

df_Investigacion4 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2019)]

In [5]:
# Reestructuración del DF para mejor uso de Pandas
df_Investigacion1 = df_Investigacion1.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion2 = df_Investigacion2.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion3 = df_Investigacion3.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion4 = df_Investigacion4.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion1 = df_Investigacion1.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

df_Investigacion2 = df_Investigacion2.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

df_Investigacion3 = df_Investigacion3.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

df_Investigacion4 = df_Investigacion4.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

In [6]:
# Renombrar las variables 2016
print(df_Investigacion1.columns)

df_Investigacion1.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Antecedentes Heredofamiliares', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', '¿Cree que los Miembros de su Equipo/Compañeros de Trabajo le Verían de Forma más Negativa Si Supieran que Padece una Enfermedad Mental?', 'Enfermedades Mentales Diagnosticadas', 'Enfermedades Mentales No Diagnosticadas', 'Puesto de Trabajo', 'Trabajo Remoto']

# Reemplazar los NaN
df_Investigacion1 = df_Investigacion1.fillna('No respondió')

Index([  1,   2,   3,   4,   5,   6,   7,   8,  10,  12,  14,  16,  17,  18,
        19,  20,  23,  24,  27,  28,  30,  31,  33,  34,  48,  49,  53,  54,
        55,  56, 114, 115, 116, 117, 118],
      dtype='int64', name='QuestionID')


In [7]:
# Renombrar las variables 2017
print(df_Investigacion2.columns)

df_Investigacion2.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Antecedentes Heredofamiliares', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', 'Importancia a la Salud Mental (Empresa)', '¿Cómo Afectó la Relación con Alguien de tu Trabajo el Haberle Contado sobre tu Enfermedad Mental?', '¿Te Identificas Abiertamente en el Trabajo como una Persona con una Enfermedad Mental?', '¿Tu Carrera Se Ha Visto Afectada por Tener una Enfermedad Mental?', 'Si Supieran Que Padeces un Trastorno Mental, ¿Cómo Crees Que Reaccionarían los Miembros de tu Equipo/Compañeros de Trabajo?', '¿Has Observado o Experimentado una Respuesta Positiva De Compañeros/Jefes al Saber que Tienes una Enfermedad Mental?', '¿En qué Medida Cree que el Sector Tecnológico Apoya a los Empleados con Enfermedades Mentales?', 'Raza']

# Reemplazar los NaN
df_Investigacion2 = df_Investigacion2.fillna('No respondió')

Index([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24,
       27, 28, 30, 31, 33, 34, 48, 49, 53, 54, 55, 56, 65, 67, 78, 79, 81, 83,
       85, 89],
      dtype='int64', name='QuestionID')


In [8]:
# Renombrar las variables 2018
print(df_Investigacion3.columns)

df_Investigacion3.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Antecedentes Heredofamiliares', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', 'Importancia a la Salud Mental (Empresa)', '¿Cómo Afectó la Relación con Alguien de tu Trabajo el Haberle Contado sobre tu Enfermedad Mental?', '¿Te Identificas Abiertamente en el Trabajo como una Persona con una Enfermedad Mental?', '¿Tu Carrera Se Ha Visto Afectada por Tener una Enfermedad Mental?', 'Si Supieran Que Padeces un Trastorno Mental, ¿Cómo Crees Que Reaccionarían los Miembros de tu Equipo/Compañeros de Trabajo?', '¿Has Observado o Experimentado una Respuesta Positiva De Compañeros/Jefes al Saber que Tienes una Enfermedad Mental?', '¿En qué Medida Cree que el Sector Tecnológico Apoya a los Empleados con Enfermedades Mentales?', 'Raza']

# Reemplazar los NaN
df_Investigacion3 = df_Investigacion3.fillna('No respondió')

Index([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24,
       27, 28, 30, 31, 33, 34, 48, 49, 53, 54, 55, 56, 65, 67, 78, 79, 81, 83,
       85, 89],
      dtype='int64', name='QuestionID')


In [9]:
# Renombrar las variables 2019
print(df_Investigacion4.columns)

df_Investigacion4.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Antecedentes Heredofamiliares', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', 'Importancia a la Salud Mental (Empresa)', '¿Cómo Afectó la Relación con Alguien de tu Trabajo el Haberle Contado sobre tu Enfermedad Mental?', '¿Te Identificas Abiertamente en el Trabajo como una Persona con una Enfermedad Mental?', '¿Tu Carrera Se Ha Visto Afectada por Tener una Enfermedad Mental?', 'Si Supieran Que Padeces un Trastorno Mental, ¿Cómo Crees Que Reaccionarían los Miembros de tu Equipo/Compañeros de Trabajo?', '¿Has Observado o Experimentado una Respuesta Positiva De Compañeros/Jefes al Saber que Tienes una Enfermedad Mental?', '¿En qué Medida Cree que el Sector Tecnológico Apoya a los Empleados con Enfermedades Mentales?', 'Raza']

# Reemplazar los NaN
df_Investigacion4 = df_Investigacion4.fillna('No respondió')

Index([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24,
       27, 28, 30, 31, 33, 34, 48, 49, 53, 54, 55, 56, 65, 67, 78, 79, 81, 83,
       85, 89],
      dtype='int64', name='QuestionID')


In [10]:
# Carpeta donde deseas guardar los archivos CSV
output_folder = r"C:\Users\DELL\OneDrive\Documentos\kadija\compu\CSV"

# Crear la carpeta si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Guardar los archivos CSV en la carpeta especificada
df_Investigacion1.to_csv(os.path.join(output_folder, 'Investigacion1.csv'), index=False)
df_Investigacion2.to_csv(os.path.join(output_folder, 'Investigacion2.csv'), index=False)
df_Investigacion3.to_csv(os.path.join(output_folder, 'Investigacion3.csv'), index=False)
df_Investigacion4.to_csv(os.path.join(output_folder, 'Investigacion4.csv'), index=False)

In [11]:
# Carga el archivo CSV en un DataFrame
df_2016 = pd.read_csv(r"C:\Users\DELL\OneDrive\Documentos\kadija\compu\CSV\Investigacion1.csv")
df_2017 = pd.read_csv(r"C:\Users\DELL\OneDrive\Documentos\kadija\compu\CSV\Investigacion2.csv")
df_2018 = pd.read_csv(r"C:\Users\DELL\OneDrive\Documentos\kadija\compu\CSV\Investigacion3.csv")
df_2019 = pd.read_csv(r"C:\Users\DELL\OneDrive\Documentos\kadija\compu\CSV\Investigacion4.csv")

In [12]:
# ¿Si Expresas que posees una Enfermedad Mental crees que en tu ambiente de Trabajo seria tomado de Mala manera?
#2016
Pregunta_52016 = df_2016['¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?'].value_counts().to_frame('Posee Enfermedad Mental Diagnosticada')

Pregunta_52016 = Pregunta_52016.rename_axis('Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental', axis=0)
Pregunta_52016 = Pregunta_52016.rename(columns={'Posee Enfermedad Mental Diagnosticada': 'Posee Enfermedad Mental Diagnosticada'})
Pregunta_52016.loc['Total'] = Pregunta_52016.sum()
print(Pregunta_52016)

#2017
Pregunta_52017 = df_2017['¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?'].value_counts().to_frame('Posee Enfermedad Mental Diagnosticada')

Pregunta_52017 = Pregunta_52017.rename_axis('Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental', axis=0)
Pregunta_52017 = Pregunta_52017.rename(columns={'Posee Enfermedad Mental Diagnosticada': 'Posee Enfermedad Mental Diagnosticada'})
Pregunta_52017.loc['Total'] = Pregunta_52017.sum()
print(Pregunta_52017)

#2018
Pregunta_52018 = df_2018['¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?'].value_counts().to_frame('Posee Enfermedad Mental Diagnosticada')

Pregunta_52018 = Pregunta_52018.rename_axis('Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental', axis=0)
Pregunta_52018 = Pregunta_52018.rename(columns={'Posee Enfermedad Mental Diagnosticada': 'Posee Enfermedad Mental Diagnosticada'})
Pregunta_52018.loc['Total'] = Pregunta_52018.sum()
print(Pregunta_52018)

# 2019
Pregunta_52019 = df_2019['¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?'].value_counts().to_frame('Posee Enfermedad Mental Diagnosticada')

Pregunta_52019 = Pregunta_52019.rename_axis('Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental', axis=0)
Pregunta_52019 = Pregunta_52019.rename(columns={'Posee Enfermedad Mental Diagnosticada': 'Posee Enfermedad Mental Diagnosticada'})
Pregunta_52019.loc['Total'] = Pregunta_52019.sum()
print(Pregunta_52019)


                                                    Posee Enfermedad Mental Diagnosticada
Respuesta Insolidaria o Mal Gestionada a un Pro...                                       
No                                                                                    341
Tal vez/No estoy seguro(a)                                                            215
Si, lo he visto                                                                       155
Si, me ha pasado                                                                       89
No respondió                                                                           39
Total                                                                                 839
                                                    Posee Enfermedad Mental Diagnosticada
Respuesta Insolidaria o Mal Gestionada a un Pro...                                       
No                                                                                    218
Tal vez/No

In [13]:
import plotly.graph_objects as go
import pandas as pd

Pregunta_52016 = pd.DataFrame({'Posee Enfermedad Mental Diagnosticada': [155, 89, 341, 215]}, index=['Sí, lo he visto', 'Sí, me ha pasado','No', 'No estoy seguro(a)'])
Pregunta_52017 = pd.DataFrame({'Posee Enfermedad Mental Diagnosticada': [83, 54, 218, 141]}, index=['Sí, lo he visto', 'Sí, me ha pasado','No','No estoy seguro(a)'])
Pregunta_52018 = pd.DataFrame({'Posee Enfermedad Mental Diagnosticada': [70, 37, 133, 72]}, index=['Sí, lo he visto','Sí, me ha pasado','No','No estoy seguro(a)'])
Pregunta_52019 = pd.DataFrame({'Posee Enfermedad Mental Diagnosticada': [39, 32, 83, 50]}, index=['Sí, lo he visto','Sí, me ha pasado','No', 'No estoy seguro(a)'])

# Combinar los df
dataframes = [
    Pregunta_52016.reset_index(),
    Pregunta_52017.reset_index(),
    Pregunta_52018.reset_index(),
    Pregunta_52019.reset_index(),
]

# Crear la lista de nombres
frames = []
for i, df in enumerate(dataframes):
    frame = go.Frame(
        data=[go.Bar(x=df['Posee Enfermedad Mental Diagnosticada'], y=df['index'], 
                     name=f'Año {2016 + i}', 
                     marker=dict(color='#BA68C8'),
                     orientation='h')],
        layout=go.Layout(
            title=f'Año {2016 + i}',
        ),
        name=f'frame{i}',
    )
    frames.append(frame)

# Crear la figura inicial
fig = go.Figure(
    data=[go.Bar(x=dataframes[0]['Posee Enfermedad Mental Diagnosticada'], y=dataframes[0]['index'], 
                 name='Año 2016', 
                 marker=dict(color='#BA68C8'),
                 orientation='h')],
    layout=go.Layout(
        title='Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental',
        xaxis_title='Cantidad',
        yaxis_title='Respuesta',
        updatemenus=[
            dict(
                type="buttons",
                direction="down",
                buttons=list([
                    dict(label="Play", method="animate", args=[None, {"frame": {"duration": 500, "fromcurrent": True}, "transition": {"duration": 300}}],
                         ),
                    dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "fromcurrent": True}, "transition": {"duration": 0}}],
                         )
                ]),
            )
        ]
    )
)

fig.update_layout(transition={'duration': 300})

fig.frames = frames

# Mostrar la figura
fig.show()

In [14]:
# Pregunta 2
# ¿Cuáles son las 5 enfermedades más comunes de los encuestados según el estado donde residen? ¿Cambiaría este ranking si se toma en cuenta la edad y el genero? 
pregunta_2 = df_2016[df_2016['Enfermedades Mentales Diagnosticadas']!= 'No respondió'].groupby('Ciudad de Residencia (EEUU)')['Enfermedades Mentales Diagnosticadas'].value_counts().nlargest(5).reset_index(name='Frecuencia')

pregunta_2 = pregunta_2.loc[:, ['Enfermedades Mentales Diagnosticadas', 'Ciudad de Residencia (EEUU)', 'Frecuencia']]  # Reordenar columnas

pregunta_2.columns = ['Enfermedades Mentales Diagnosticadas', 'Ciudad de Residencia (EEUU)', 'Cantidad de personas']  # Renombrar columnas

print(pregunta_2.to_string(header=True, index=False))

                               Enfermedades Mentales Diagnosticadas Ciudad de Residencia (EEUU)  Cantidad de personas
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                  California                    34
Trastorno del Estado del Ánimo (Depresión, Trastorno Bipolar, etc.)                  California                    19
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                    Illinois                    18
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                 Pensilvania                    16
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                  Nueva York                    13


In [15]:
data = pd.DataFrame({
    'State': ['California', 'California', 'Illinois', 'Pennsylvania', 'New York'],
    'Frequency': [34, 19, 18, 16, 13]
})


data = data.groupby('State')['Frequency'].sum().reset_index()


data['State Abbrev'] = data['State'].map({
    'California': 'CA',
    'Illinois': 'IL',
    'Pennsylvania': 'PA',
    'New York': 'NY'
})


data['Mental Health Condition'] = data['State'].map({
    'California': 'Trastorno de Ansiedad (Generalizado,Fobia, Social, Trastorno del Ánimo (Depresión, Trastorno Bipolar, etc))',
    'Illinois': 'Generalizado,Fobia, Social',
    'Pennsylvania': 'Generalizado,Fobia, Social',
    'New York': 'Generalizado,Fobia, Social'
})


fig = px.choropleth(
    data,
    locations="State Abbrev", 
    locationmode="USA-states",
    color="Frequency",
    scope="usa",
    color_continuous_scale="Purples", 
    hover_name="State",
    hover_data=["Frequency", "Mental Health Condition"], 
    color_continuous_midpoint=data['Frequency'].mean(), 
    title="Frecuencia de Condiciones de Salud Mental por Estado en EE. UU.",  
)

fig.update_layout(coloraxis_colorbar=dict(title="Frecuencia de Casos"))

# Show the graph
fig.show()

NameError: name 'px' is not defined

In [ ]:
data = pd.DataFrame({
    'State': ['Alabama', 'Arizona', "California","Carolina del Norte", "Carolina del Sur", "Connecticut", "Delaware", "Florida","Georgia", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana","Maine","Maryland","Massachusetts","Michigan","Minnesota","Missouri","Nebraska","Nueva Jersey","Nueva York","Nuevo Hampshire","Ohio","Oklahoma","Oregon","Pensilvania","Rhode Island","Tennessee","Texas","Utah","Vermont","Virginia","Virginia del Oeste","Washington","Wisconsin"],
    'Frequency': [2,1,34,8,1,3,1,5,3,2,18,8,2,2,2,2,3,4,12,12,2,2,3,13,1,6,2,11,16,1,6,13,3,1,4,1,10,5]
})

# Group the data by state and sum the frequencies
data = data.groupby('State')['Frequency'].sum().reset_index()

# Create a column with state abbreviations
data['State Abbrev'] = data['State'].map({
    "Alabama":"AL",
"Arizona": "AZ",
"California": "CA",
"Carolina del Norte": "NC",
"Carolina del Sur":"SC",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Missouri": "MO",
"Nebraska": "NE",
"Nueva Jersey": "NJ",
"Nueva York": "NY",
"Nuevo Hampshire": "NH",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pensilvania": "PA",
"Rhode Island": "RI",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Virginia del Oeste": "WV",
"Washington": "WA",
"Wisconsin": "WI"

})

# Add a new column with mental health condition information
data['Mental Health Condition'] = data['State'].map({
   "Alabama":"Trastorno del Estado del Ánimo (Depresion, Trastorno Bipolar, etc.)",
"Arizona": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"California": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Carolina del Norte": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Carolina del Sur":"Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Connecticut": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Delaware": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Florida": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Georgia": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Idaho": "Trastorno del Estado del Ánimo (Depresion, Trastorno Bipolar, etc.)",
"Illinois": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Indiana": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Kansas": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Kentucky": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Louisiana": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Maine": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Maryland": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Massachusetts": "Trastorno del Estado del Ánimo (Depresion, Trastorno Bipolar, etc.)",
"Michigan": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Minnesota": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Missouri": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Nebraska": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Nueva Jersey": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Nueva York": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Nuevo Hampshire": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Ohio": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Oklahoma": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Oregon": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Pensilvania": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Rhode Island": "Trastorno Obsesivo-Compulsivo",
"Tennessee": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Texas": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Utah": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Vermont": "Trastorno de Consumo de Sustancias",
"Virginia": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Virginia del Oeste": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Washington": "Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)",
"Wisconsin": "Trastorno del Estado del Ánimo (Depresion, Trastorno Bipolar, etc.)"
})

# Create a choropleth map
fig = px.choropleth(
    data,
    locations="State Abbrev",  
    locationmode="USA-states",
    color="Frequency",
    scope="usa",
    color_continuous_scale=[
        (0, "lightgray"), 
        (0.2, "lightblue"),  
        (0.4, "skyblue"),  
        (0.6, "blue"), 
        (0.8, "darkblue"),  
        (1, "navy") 
    ],
    hover_name="State",
    hover_data=["Frequency", "Mental Health Condition"],  
    color_continuous_midpoint=data['Frequency'].mean(),  
    title="Frecuencia de Condiciones de Salud Mental por Estado en EE. UU.", 
)

fig.update_layout(coloraxis_colorbar=dict(title="Frecuencia de Casos"))

In [ ]:
# Parte 2, Pregunta 2
df_20162=df_2016
# Convertir la columna Edad a tipo numérico, convirtiendo valores no numéricos a NaN
df_20162['Edad'] = pd.to_numeric(df_20162['Edad'], errors='coerce')

# Eliminar filas con valores NaN en la columna Edad
df_20162 = df_20162.dropna(subset=['Edad'])

# Convertir la columna Edad a tipo entero
df_20162['Edad'] = df_20162['Edad'].astype(int)

# Convertir la columna Enfermedades Mentales Diagnosticadas a tipo string
df_20162['Enfermedades Mentales Diagnosticadas'] = df_20162['Enfermedades Mentales Diagnosticadas'].astype(str)

# Crear una columna Edad_binned con los rangos de edad
df_20162['Edad'] = pd.cut(df_20162['Edad'], bins=[18, 25, 32, 38, 44, 50, 56, 66], 
                                 labels=['19-25', '26-32', '33-38', '39-44', '45-50', '51-56', '57-66'],
                                 include_lowest=True)

# Eliminar filas con "No respondio" en la columna Enfermedades Mentales Diagnosticadas
df_20162 = df_20162[~df_20162['Enfermedades Mentales Diagnosticadas'].isin(['No respondió'])]

# Agrupar por Edad y Enfermedades Mentales Diagnosticadas, y contar la frecuencia
pregunta2_2 = df_20162.groupby(['Edad', 'Enfermedades Mentales Diagnosticadas']).size().reset_index(name='Cantidad de personas')

# Seleccionar la enfermedad mental más frecuente para cada rango de edad
pregunta2_2 = pregunta2_2.loc[pregunta2_2.groupby('Edad')['Cantidad de personas'].idxmax()]

# Reordenar las columnas
pregunta2_2 = pregunta2_2[['Enfermedades Mentales Diagnosticadas', 'Edad', 'Cantidad de personas']]

# Mostrar la tabla con la enfermedad mental más frecuente para cada rango de edad
print(pregunta2_2)

                 Enfermedades Mentales Diagnosticadas   Edad  \
8   Trastorno de Ansiedad (Genralizado, Social, Fo...  19-25   
22  Trastorno de Ansiedad (Genralizado, Social, Fo...  26-32   
36  Trastorno de Ansiedad (Genralizado, Social, Fo...  33-38   
50  Trastorno de Ansiedad (Genralizado, Social, Fo...  39-44   
64  Trastorno de Ansiedad (Genralizado, Social, Fo...  45-50   
78  Trastorno de Ansiedad (Genralizado, Social, Fo...  51-56   
92  Trastorno de Ansiedad (Genralizado, Social, Fo...  57-66   

    Cantidad de personas  
8                     37  
22                    75  
36                    65  
50                    28  
64                    14  
78                     5  
92                     4  


C:\Users\DELL\AppData\Local\Temp\ipykernel_22256\2308210389.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20162['Edad'] = df_20162['Edad'].astype(int)
C:\Users\DELL\AppData\Local\Temp\ipykernel_22256\2308210389.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20162['Enfermedades Mentales Diagnosticadas'] = df_20162['Enfermedades Mentales Diagnosticadas'].astype(str)
C:\Users\DELL\AppData\Local\Temp\ipykernel_22256\2308210389.py:16: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
# Parte 3, pregunta 2
Pregunta2_3 = df_Investigacion.groupby(['Enfermedades Mentales Diagnosticadas', 'Género']).size().reset_index(name='Cantidad de personas')

Pregunta2_3 = Pregunta2_3[Pregunta2_3['Género']!= 'No respondió']  # Eliminar "No respondió" de la columna "Género"

top_5_enfermedades = Pregunta2_3.groupby('Enfermedades Mentales Diagnosticadas')['Cantidad de personas'].sum().nlargest(6).index

Pregunta2_3 = Pregunta2_3[Pregunta2_3['Enfermedades Mentales Diagnosticadas'].isin(top_5_enfermedades)]

Pregunta2_3 = Pregunta2_3[Pregunta2_3['Enfermedades Mentales Diagnosticadas']!= 'No respondió']

Pregunta2_3 = Pregunta2_3.pivot_table(index='Enfermedades Mentales Diagnosticadas', columns='Género', values='Cantidad de personas', fill_value=0)

print(Pregunta2_3)